<center><h2>Personalized Recommendations for Improving Predicted Sleep Quality
    </h2></center>

<p align=”justify”>This notebook illustrates the code logic in making user-level recommendations based on our random forest model and the resulting output that is used to generate visualizations within the <a href = "https://bioloopsleep.com/demo">Bioloop Sleep</a> user profile</p>

<p align=”justify”>We make perturbations (tweaks) to individual input features, ceteris paribus, that can be transcribed into actionable user behavior to showcase how hypothetical behavioral changes influence an individual's predicted sleep quality outcome. We used information from Oura's <a href = "https://cloud.ouraring.com/docs/">API</a> to transcribe these perturbations into intelligible recommendations. For instance, per the Oura API a "latency of about 15 minutes (900 seconds) gives best [sleep] score" and a training frequency "value is 95 when the user has got more than 100 minutes of medium or high intensity activity on at least three days during past seven days." In other cases, we set the perturbations based on how we engineered the feature set. For example, activity routine was a feature we developed where the ideal routine score is 0.</p>

#### Reading in the Model and User Data

In [1]:
#loading our random forest model
from joblib import dump, load
rf_mod = load('predicted_sleep_score_model.joblib')

In [2]:
#reading in user data (after transformations)
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
df_gold = pd.read_csv('data_ingestion/29112019/unzipped_data/gold.csv',parse_dates = ['summary_date'])

In [3]:
#selecting features used in our model to filter those features from our user base
model_columns = ['summary_date','rol_sleep_afterMidnight_7d','rol_onset_latency_7d',
                 'rol_bedtime_start_delta_7d','rol_steps_7d','rol_is_traveling_7d',
                 'score_training_frequency', 'score_training_volume',
                 'avg_sleep_score_next_week','activity_routine_score','sleep_routine_score',
                 'experiment_category_Chamomile Tea','experiment_category_Magnesium',
                 'experiment_category_Meditation','rol_score_disturbances_7d','rol_sleep_score_7d'
                ]

features = [f for f in model_columns if f not in {'summary_date',
                                                  'avg_sleep_score_next_week'}]



#### Example Recommendations for Increasing Predicted Sleep Score over the Next Week<br>Using Data from 3 Different Users on Separate Dates


In [2]:
# selecting 3 different users on different dates for recommendations

user_A = df_gold[(df_gold['user_id_TE2CPSSWP4QUGFAJQZ5FHITIKPNCCICX'] == 1) & (df_gold['summary_date'] == '2019-09-30')]\
.filter(features)

user_B = df_gold[(df_gold['user_id_UZNLIVG56OJ2YYIUWHQZ4IOPJ7YBSEOE'] == 1) & (df_gold['summary_date'] == '2019-10-07')]\
.filter(features)

user_C = df_gold[(df_gold['user_id_PQC6APVY4RN6ZBRKEOMFMGJDPCS67GDY'] == 1) & (df_gold['summary_date'] == '2019-10-15')]\
.filter(features)

In [5]:
def nominal_recommender(model,user_data):
    from copy import deepcopy
    '''returns a dataframe in the format of feature, adjusted odds of improving avg sleep score in next 7 days
    and the description of how to do so'''
    
    actionable_features = ['rol_bedtime_start_delta_7d','sleep_routine_score','activity_routine_score',
                        'score_training_frequency','score_training_volume',
                        'rol_steps_7d','rol_sleep_afterMidnight_7d','rol_onset_latency_7d',
                        'experiment_category_Chamomile Tea','experiment_category_Magnesium',
                        'experiment_category_Meditation']
    
    
    descriptions = {
    'sleep_routine_score':
    "stay consistent with your sleep routines: go to sleep around the same time and get the same amount of sleep each night",
    'activity_routine_score': 
    "stay consistent with your activity routines: maintain the same frequency and intensity of activity from week to week",
    'rol_score_disturbances_7d':"limit the number of times you get up and how frequently you move throughout your sleep over the next week",
    'score_training_frequency': "get more than 100 minutes of medium or high intensity activity on at least three of the next seven days",
    'score_training_volume':"attain a sum total of 750 MET minutes over the next 7 days",
    'rol_bedtime_start_delta_7d': "limit the weekly variance in the times that you fall asleep",
    'rol_steps_7d': " get an additional 2000 steps in each day over the next week",
    'rol_sleep_afterMidnight_7d': "consistently go to bed before midnight each night over the next week",
    'rol_onset_latency_7d': "aim to fall asleep on average within 15 minutes after laying in bed",
    'user baseline': "your predicted avg sleep score over the next 7 days",
    'experiment_category_Chamomile Tea': "experiment in drinking Chamomile Tea",
    'experiment_category_Magnesium': "experiment in taking Magnesium supplements",
    'experiment_category_Meditation': "experiment with Meditation"    
    }
    
    
    base_score = model.predict(user_data)[0]
    adj_scores = []
    for a in actionable_features:
        #reset the copy on each iteration
        user_data_copy = user_data.copy()
        if a in {'rol_bedtime_start_delta_7d','sleep_routine_score','activity_routine_score'}:
            #all features above have an ideal target of 0
            user_data_copy[a] = 0
            new_score = model.predict(user_data_copy)[0]
            adj_scores.append([a,new_score])

        elif a in {'rol_score_disturbances_7d','score_training_frequency','score_training_volume'}:
            #all features above have an ideal target of 95+
            user_data_copy[a] = 95
            new_score = model.predict(user_data_copy)[0]
            adj_scores.append([a,new_score])
        elif a in {'rol_steps_7d'}:
            #1 mile is approximately 2000 steps (will increment avg by 2000)
            user_data_copy[a] = user_data_copy[a].values[0] + 2000
            new_score = model.predict(user_data_copy)[0]
            adj_scores.append([a,new_score])

        elif a == 'rol_sleep_afterMidnight_7d':
            if user_data_copy[a].values[0] > 0:
                user_data_copy[a] = 0
                new_score = model.predict(user_data_copy)[0]
                adj_scores.append([a,new_score])
        elif a == 'rol_onset_latency_7d':
            user_data_copy[a] = 900    
            new_score = model.predict(user_data_copy)[0]
            adj_scores.append([a,new_score])
                    
        elif a == 'experiment_category_Chamomile Tea':
            if user_data[a].values[0] == 0:
                user_data_copy[a] = 1    
                new_score = model.predict(user_data_copy)[0]
                adj_scores.append([a,new_score])

        elif a == 'experiment_category_Magnesium':
            if user_data[a].values[0] == 0:
                user_data_copy[a] = 1    
                new_score = model.predict(user_data_copy)[0]
                adj_scores.append([a,new_score])            
            
        elif a == 'experiment_category_Meditation':
            if user_data[a].values[0] == 0:
                user_data_copy[a] = 1    
                new_score = model.predict(user_data_copy)[0]
                adj_scores.append([a,new_score])

            
    adj_scores.append(['user baseline',base_score])
    
    description_list = []
    for k,v in descriptions.items():
        description_list.append([k,v])

    descript_df = pd.DataFrame(description_list,columns = ['Feature','Description'])
    
    score_df = pd.DataFrame(adj_scores,
            columns = ['Feature','Predicted Future Avg Sleep Score'])
    
    return score_df.merge(descript_df,on='Feature',how = 'left')

#### Output Used to Generate Visuals in the Biosleep UI

In [6]:
#calling function for recommendation output
nominal_recommender(model = rf_mod,user_data = user_A)

,Feature,Predicted Future Avg Sleep Score,Description
0,rol_bedtime_start_delta_7d,74.322857,limit the weekly variance in the times that yo...
1,sleep_routine_score,74.458571,stay consistent with your sleep routines: go t...
2,activity_routine_score,74.015714,stay consistent with your activity routines: m...
3,score_training_frequency,73.565714,get more than 100 minutes of medium or high in...
4,score_training_volume,72.881429,attain a sum total of 750 MET minutes over the...
5,rol_steps_7d,73.484286,get an additional 2000 steps in each day over...
6,rol_onset_latency_7d,74.400000,aim to fall asleep on average within 15 minute...
7,experiment_category_Chamomile Tea,73.608571,experiment in drinking Chamomile Tea
8,experiment_category_Magnesium,73.578571,experiment in taking Magnesium supplements
9,experiment_category_Meditation,73.742857,experiment with Meditation


In [7]:
nominal_recommender(model = rf_mod,user_data = user_B)

,Feature,Predicted Future Avg Sleep Score,Description
0,rol_bedtime_start_delta_7d,64.272857,limit the weekly variance in the times that yo...
1,sleep_routine_score,63.648571,stay consistent with your sleep routines: go t...
2,activity_routine_score,64.038571,stay consistent with your activity routines: m...
3,score_training_frequency,63.612857,get more than 100 minutes of medium or high in...
4,score_training_volume,63.617143,attain a sum total of 750 MET minutes over the...
5,rol_steps_7d,67.481429,get an additional 2000 steps in each day over...
6,rol_sleep_afterMidnight_7d,63.175714,consistently go to bed before midnight each ni...
7,rol_onset_latency_7d,63.452857,aim to fall asleep on average within 15 minute...
8,experiment_category_Chamomile Tea,63.814286,experiment in drinking Chamomile Tea
9,experiment_category_Magnesium,63.924286,experiment in taking Magnesium supplements


In [8]:
nominal_recommender(model = rf_mod,user_data = user_C)

,Feature,Predicted Future Avg Sleep Score,Description
0,rol_bedtime_start_delta_7d,72.995714,limit the weekly variance in the times that yo...
1,sleep_routine_score,72.592857,stay consistent with your sleep routines: go t...
2,activity_routine_score,73.888571,stay consistent with your activity routines: m...
3,score_training_frequency,72.780000,get more than 100 minutes of medium or high in...
4,score_training_volume,72.185714,attain a sum total of 750 MET minutes over the...
5,rol_steps_7d,73.144286,get an additional 2000 steps in each day over...
6,rol_onset_latency_7d,73.605714,aim to fall asleep on average within 15 minute...
7,experiment_category_Chamomile Tea,72.785714,experiment in drinking Chamomile Tea
8,experiment_category_Magnesium,72.674286,experiment in taking Magnesium supplements
9,experiment_category_Meditation,73.027143,experiment with Meditation
